In [ ]:
%pip install openai
%pip install ffmpeg-python

In [ ]:
#import json
#%cd path/to/file/on/colab
import pandas as pd
#import numpy as np
import sys
import os
import shutil
#sys.path.insert(0,'/Users/sanjivjha/IMP-OIC/')
from utils.extractframes import FrameExtractor
import graphene
from inference.gpt_ask import run_gpt
directory_path = 'eval/quant'


In [ ]:
# Run OIC function to create prompt for LLM and save it in out folder as a text file
def run_oic():
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    for video in os.scandir(directory_path):
        if video.is_file():
            ex = FrameExtractor(directory_path+'/'+video.name, 1, 5)
            ex.main()
            g = graphene.Graphene(0.3, 0.6)

            if not os.path.isdir(OUT_DIR):
                os.mkdir(OUT_DIR)
            text = os.path.splitext(video.name)[0]+'graph2text.txt'
            img_path = directory_path+'/'+os.path.splitext(video.name)[0]+'-opencv'


            g.classify_images_window(img_path, 5)
            graph_path = img_path + "/img/JSON"
            g.generate_temporal_graph_frames(graph_path, img_path + "/img")

            g.tg.to_text(os.path.join(OUT_DIR, text))

            if os.path.isfile(os.path.join(OUT_DIR,text)):
                with open(os.path.join(OUT_DIR,text)) as f:
                    context = "".join(map(str,f.readlines()))
                    return str(context)

In [ ]:
# Run OIC and ask an example question to GPT3.5
if os.path.isdir(directory_path):
  prompt = run_oic()
  if prompt is not None:
    answer = run_gpt(prompt, 'What is happening in the scene?')
    print('OIC answer: {}'.format(answer))
  else:
    print('OIC answer: {}'.format(None))

In [ ]:
# Remove temp folder with all the content inside it for next execution
import shutil

shutil.rmtree('temp')


In [ ]:
# Ask question to GPT3.5-16k based on generated/any other prompt
with open(os.path.join('/path/to/out/folder/and/prompt/IMG_2358graph2text.txt')) as f:
      context = "".join(map(str,f.readlines()))

      answer = run_gpt(context, 'enter your question')
      print('OIC answer: {}'.format(answer))
